In [31]:
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.cm as cmx

In [32]:
clusters = pd.read_json("./files/cluster.json")

ValueError: Expected object or value

In [33]:
def plotSteps(data):
    for runIdx in range(0, len(data)):
        fig, ax = plt.subplots()
        iteration = data.iloc[runIdx]["iteration"]
        clusters = data.iloc[runIdx]["clusters"]
        norm = colors.Normalize(vmin=0, vmax=len(clusters))
        print("iteration: %s, clusters: %s" % (iteration, len(clusters)))
        for cIdx in range(0, len(clusters)):
            rgba_color = cm.gnuplot(norm(cIdx))
            pointsX = [point[0] for point in clusters[cIdx]["points"]]
            pointsY = [point[1] for point in clusters[cIdx]["points"]]
            ax.scatter(pointsX, pointsY, color=rgba_color)
            ax.scatter(clusters[cIdx]["centroid"][0], clusters[cIdx]["centroid"][1], color=rgba_color, marker="+")

        plt.show()

In [34]:
plotSteps(clusters)

NameError: name 'clusters' is not defined

In [35]:
def plot(data):
    for runIdx in range(0, len(data)):
        fig, ax = plt.subplots()
        clusters_step1 = data.iloc[runIdx]["steps"][0]["clusters"]
        clusters_step2 = data.iloc[runIdx]["steps"][1]["clusters"]
        norm = colors.Normalize(vmin=0, vmax=len(clusters_step1))
        for cIdx in range(0, len(clusters_step1)):
            rgba_color = cm.gnuplot(norm(cIdx))
            ax.plot(clusters_step1[cIdx]["syntheticCenter"], "--", color=rgba_color, label="cluster %s" % cIdx)
            ax.plot(clusters_step2[cIdx]["syntheticCenter"], color=rgba_color, label="cluster %s - rescheduled" % cIdx)
            print(clusters_step2[cIdx]["metric"])
        ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)

        ax.set(xlabel='Hours', ylabel='Power (kW)',title='')
        plt.show()

In [36]:
def plot_aggm(summary, file_name=None): 
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. agg m"], "--", color=rgba_color, label="s1. agg m")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. agg m"], color=rgba_color, label="s2. agg m")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Aggregate PAR',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig
    
def plot_maxm(summary, file_name=None): 
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. max m"], "--", color=rgba_color, label="s1. max m")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. max m"], color=rgba_color, label="s2. max m")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Max. Aggregate PAR',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig
    
def plot_peak(summary, file_name=None):
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. peak"], "--", color=rgba_color, label="s1. peak")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. peak"], color=rgba_color, label="s2. peak")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Peak power (kW)',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig

# PAR Aggregate

In [37]:
summaryBatch_global = pd.read_json("./files/summaryBatch.json")
#summaryBatch_global = summaryBatch_global[["k","clusters","s1. agg m","s1. max m","s1. peak","s2. agg m","s2. max m","s2. peak","total m"]]
#batch_global = pd.read_json("./files/batch.json")

k = 6
sampleSizeMin = 0.8
sampleSizeMax = 1.0

def sampleSizeRange():
    return (x/10 for x in range(int(sampleSizeMin * 10), int((sampleSizeMax * 10) + 1)))

In [38]:
index_list = []
step_list = []

for i in range(0 , len(summaryBatch_global)):
    register = summaryBatch_global.iloc[i]
    
    crossfold = json_normalize(register["crossfold"])
    splits = crossfold["splits"].values[0]
    steps = summaryBatch_global.iloc[i]["step"]
    for j in range(0, splits):
        step = json_normalize(steps[j])
        for ki in range(1, k+1):
            index_list.append((float("{0:.2f}".format(crossfold["sampleSize"].values[0])), j, ki-1))
            step_list.append(step.iloc[ki-1])
        
    
json_normalize(summaryBatch_global.iloc[0]["crossfold"])
json_normalize(summaryBatch_global.iloc[0]["step"][0])

index = pd.MultiIndex.from_tuples(index_list, names=["sampleSize", "split", "k"])
summaryBatch_global_treated = pd.DataFrame(step_list, index=index)
summaryBatch_global_treated

clusters  k  s1. agg m  s1. max m  \
sampleSize split k                                                      
0.8        0     0                     [151]  1   1.424382   1.424382   
                 1                  [76, 75]  2   1.003973   1.480372   
                 2              [51, 50, 50]  3   1.027410   1.548760   
                 3          [37, 37, 37, 40]  4   1.105194   1.575134   
                 4      [30, 30, 30, 30, 31]  5   1.052351   1.566398   
                 5  [25, 25, 25, 26, 25, 25]  6   1.135258   1.726891   
           1     0                     [151]  1   1.314654   1.314654   
                 1                  [76, 75]  2   1.008094   1.386131   
                 2              [50, 51, 50]  3   1.026511   1.416727   
                 3          [37, 37, 37, 40]  4   1.089510   1.567497   
                 4      [30, 30, 31, 30, 30]  5   1.049048   1.502706   
                 5  [25, 25, 25, 26, 25, 25]  6   1.057027   1.536275   
0.9        0     0                     [170]  1   1.353494   1.353494   
                 1                  [85, 85]  2   1.001997   1.458291   
                 2              [57, 57, 56]  3   1.020559   1.457843   
                 3          [42, 43, 42, 43]  4   1.045273   1.465376   
                 4      [34, 34, 34, 34, 34]  5   1.063893   1.516707   
                 5  [28, 28, 28, 28, 29, 29]  6   1.098609   1.705600   
           1     0                     [170]  1   1.365663   1.365663   
                 1                  [85, 85]  2   1.006789   1.490554   
                 2              [58, 56, 56]  3   1.035460   1.426533   
                 3          [42, 43, 43, 42]  4   1.083962   1.483545   
                 4      [34, 34, 34, 34, 34]  5   1.093977   1.664607   
                 5  [28, 28, 28, 28, 29, 29]  6   1.118916   1.676021   
1.0        0     0                     [189]  1   1.371114   1.371114   
                 1                  [95, 94]  2   1.012142   1.385406   
                 2              [63, 63, 63]  3   1.043036   1.501961   
                 3          [47, 47, 47, 48]  4   1.103008   1.623300   
                 4      [37, 37, 37, 38, 40]  5   1.072332   1.531785   
                 5  [31, 31, 31, 31, 32, 33]  6   1.137990   1.699409   
           1     0                     [189]  1   1.371114   1.371114   
                 1                  [95, 94]  2   1.001534   1.415408   
                 2              [63, 63, 63]  3   1.033883   1.473362   
                 3          [47, 47, 47, 48]  4   1.138016   1.652741   
                 4      [37, 37, 37, 38, 40]  5   1.055651   1.509161   
                 5  [31, 31, 31, 31, 32, 33]  6   1.171027   1.761491   

                      s1. peak   total m  
sampleSize split k                        
0.8        0     0  566.965183  1.424382  
                 1  304.230783  1.424382  
                 2  209.166283  1.424382  
                 3  250.559567  1.424382  
                 4  141.296083  1.424382  
                 5  125.289450  1.424382  
           1     0  503.514067  1.314654  
                 1  276.535733  1.314654  
                 2  186.850100  1.314654  
                 3  235.003867  1.314654  
                 4  126.919350  1.314654  
                 5  109.949167  1.314654  
0.9        0     0  591.066433  1.353494  
                 1  334.293500  1.353494  
                 2  219.970183  1.353494  
                 3  161.536200  1.353494  
                 4  178.568717  1.353494  
                 5  150.030200  1.353494  
           1     0  595.843883  1.365663  
                 1  336.585600  1.365663  
                 2  215.538567  1.365663  
                 3  172.390433  1.365663  
                 4  224.340300  1.365663  
                 5  174.251717  1.365663  
1.0        0     0  676.160150  1.371114  
                 1  389.438233  1.371114  
                 2  275.758733  1.371114  
              

In [39]:
k2 = 2
query = f"k == {k2} and sampleSize == {sampleSizeMin}"
summaryBatch_global_treated.query(query)["s1. agg m"].mean()
summaryBatch_global_treated.query(query)["s1. max m"].mean()
summaryBatch_global_treated.query(query)["s1. peak"].std()

19.583357660300479

In [40]:
with open('summaryBatch.tex','w') as tf:
    tf.write(summaryBatch_global.to_latex())

In [41]:
index_list = []
step_list = []

for j in range(1, k+1):
    for ssize in sampleSizeRange():
        
        index_list.append((ssize, "mean", j))
        query = f"k == {j} and sampleSize == {ssize}"
        data = {"s1. agg m" : summaryBatch_global_treated.query(query)["s1. agg m"].mean(),
                "s1. max m" : summaryBatch_global_treated.query(query)["s1. max m"].mean(), 
                "s1. peak" : summaryBatch_global_treated.query(query)["s1. peak"].mean()}
        s = pd.Series(data,index=["s1. agg m","s1. max m","s1. peak"])
        step_list.append(s)
        #print(f"sampleSize = {ssize}, k = {j}")
        
        index_list.append((ssize, "stdv", j))
        query = f"k == {j} and sampleSize == {ssize}"
        data = {"s1. agg m" : summaryBatch_global_treated.query(query)["s1. agg m"].std(),
                "s1. max m" : summaryBatch_global_treated.query(query)["s1. max m"].std(), 
                "s1. peak" : summaryBatch_global_treated.query(query)["s1. peak"].std()}
        s = pd.Series(data,index=["s1. agg m","s1. max m","s1. peak"])
        step_list.append(s)
        
index = pd.MultiIndex.from_tuples(index_list, names=["sampleSize", "metric", "k"])
summaryBatch_global_statistics = pd.DataFrame(step_list, index=index)
summaryBatch_global_statistics

s1. agg m  s1. max m    s1. peak
sampleSize metric k                                  
0.8        mean   1   1.369518   1.369518  535.239625
           stdv   1   0.077589   0.077589   44.866715
0.9        mean   1   1.359579   1.359579  593.455158
           stdv   1   0.008605   0.008605    3.378167
1.0        mean   1   1.371114   1.371114  676.160150
           stdv   1   0.000000   0.000000    0.000000
0.8        mean   2   1.006033   1.433252  290.383258
           stdv   2   0.002914   0.066638   19.583358
0.9        mean   2   1.004393   1.474423  335.439550
           stdv   2   0.003388   0.022813    1.620759
1.0        mean   2   1.006838   1.400407  390.920225
           stdv   2   0.007501   0.021214    2.095853
0.8        mean   3   1.026961   1.482743  198.008192
           stdv   3   0.000635   0.093362   15.779925
0.9        mean   3   1.028009   1.442188  217.754375
           stdv   3   0.010537   0.022139    3.133626
1.0        mean   3   1.038459   1.487661  283.908767
           stdv   3   0.006472   0.020223   11.525888
0.8        mean   4   1.097352   1.571316  242.781717
           stdv   4   0.011091   0.005400   10.999541
0.9        mean   4   1.064617   1.474460  166.963317
           stdv   4   0.027358   0.012848    7.675102
1.0        mean   4   1.120512   1.638021  325.973233
           stdv   4   0.024755   0.020818    5.959873
0.8        mean   5   1.050700   1.534552  134.107717
           stdv   5   0.002336   0.045037   10.165886
0.9        mean   5   1.078935   1.590657  201.454508
           stdv   5   0.021272   0.104581   32.365397
1.0        mean   5   1.063992   1.520473  252.382500
           stdv   5   0.011795   0.015998    1.520775
0.8        mean   6   1.096142   1.631583  117.619308
           stdv   6   0.055317   0.134786   10.847218
0.9        mean   6   1.108763   1.690810  162.140958
           stdv   6   0.014359   0.020915   17.127199
1.0        mean   6   1.154509   1.730450  231.257458
           stdv   6   0.023361   0.043899    5.232390

In [42]:
def getMetrics(metric, summaryMetric):

    index_list = [sampleSizeRange()]
    cols = [x for x in range(1, k+1)]
    step_list = []

    for ssize in sampleSizeRange():
        query = f"sampleSize == {ssize} and metric == \"{summaryMetric}\" "
        dataAllK = summaryBatch_global_statistics.query(query)[f"{metric}"]
        dictAllK = {}
        for j in range(1, k+1):
            d = dataAllK.reset_index().query(f"k == {j}")[f"{metric}"]
            dictAllK[f"{j}"] = d.values
        s = pd.Series(dictAllK, name=f"{ssize}")
        step_list.append(s)

    return pd.DataFrame(step_list)
    
getMetrics("s1. agg m", "mean")
getMetrics("s1. agg m", "stdv")

,1,2,3,4,5,6
0.8,[0.077589382358],[0.00291391769277],[0.000635335635322],[0.0110906619927],[0.0023361277409],[0.0553171041863]
0.9,[0.00860474420191],[0.00338807729406],[0.0105367982002],[0.0273575926136],[0.0212719240479],[0.0143586155419]
1.0,[0.0],[0.00750056793239],[0.00647215792185],[0.0247545505461],[0.0117950898263],[0.0233605043322]


In [43]:
series = [pd.Series(np.random.rand(3), name=c) for c in list('abcdefg')]
series
pd.DataFrame.from_items([(s.name, s) for s in series])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  This is separate from the ipykernel package so we can avoid doing imports until


,a,b,c,d,e,f,g
0,0.214750,0.967157,0.385282,0.143497,0.015556,0.552432,0.346895
1,0.464562,0.306218,0.494906,0.128952,0.652576,0.780512,0.027585
2,0.777856,0.855024,0.086775,0.954191,0.591944,0.256130,0.759564
